In [1]:
import trlib
# import autograd.numpy as np
# from autograd import grad, hessian, hessian_vector_product

In [3]:
import scipy.sparse
import numpy as np
from scipy.sparse.linalg import LinearOperator

H = scipy.sparse.diags(np.linspace(-1.0, 100.0, 1000),0)
g = np.ones(1000)
x, TR = trlib.trlib_solve(H, g, 1.0)
np.linalg.norm(x)
1.0000000000000002
x, TR = trlib.trlib_solve(H, g, .5, reentry=True, TR=TR)
0.50000000000000011

H_lo = LinearOperator(H.shape, matvec=lambda vec: H * vec)
x, TR = trlib.trlib_solve(H_lo, g, 1.0)
np.linalg.norm(x)


1.0000000000000004

In [10]:
h = np.random.random((5, 5))
h = np.matmul(h, h.transpose()) + np.eye(5)

def f(x):
    return 0.5 * np.dot(x, np.matmul(h, x))

def g(x):
    return np.matmul(h, x)

def hvp(x, g):
    return np.matmul(h, g)

trlib.umin(f, g, hvp, np.random.random(5))


it   obj         ‖g‖        radius     step       rho          ?  nhv
   0 +4.4772e+00 8.2341e+00 4.4721e-01 4.4721e-01 +1.0000e+00  +    1
   1 +1.5687e+00 4.7778e+00 8.9443e-01 6.2189e-01 +1.0000e+00  +    1
   2 +8.3092e-02 4.7673e-01 1.7889e+00 3.4147e-01 +1.0000e+00  +    2
   3 +2.3281e-03 9.3241e-02 3.5777e+00 5.5340e-02 +1.0000e+00  +    3
   4 +1.0250e-06 1.6006e-03 7.1554e+00 1.2866e-03 +1.0000e+00  +    4
   5 +1.9722e-14 1.9997e-07


array([  9.57325040e-08,  -1.05325259e-07,   5.31485231e-08,
        -1.23588008e-07,   2.34877964e-08])

In [15]:
import autograd
import autograd.numpy as np


def f(x):
    return 0.5 * np.dot(x, np.matmul(h, x))

g = autograd.grad(f)
hvp = autograd.hessian_vector_product(f)

trlib.umin(f, g, hvp, np.random.random(5))


it   obj         ‖g‖        radius     step       rho          ?  nhv
   0 +2.7673e+00 6.5189e+00 4.4721e-01 4.4721e-01 +1.0000e+00  +    1
   1 +6.2800e-01 3.0519e+00 8.9443e-01 3.9558e-01 +1.0000e+00  +    1
   2 +2.4369e-02 2.4871e-01 1.7889e+00 1.9172e-01 +1.0000e+00  +    2
   3 +6.6650e-04 4.9853e-02 3.5777e+00 3.0320e-02 +1.0000e+00  +    3
   4 +2.4716e-09 7.2302e-05 7.1554e+00 6.8655e-05 +1.0000e+00  +    5
   5 +5.8903e-41 2.2313e-20


array([ -1.69406589e-21,   6.77626358e-21,   0.00000000e+00,
         0.00000000e+00,   1.69406589e-21])

In [19]:
class objective(object):
    def __init__(self):
#         h = np.random.random((5, 5))
#         h = np.matmul(h, h.transpose()) + np.eye(5)
#         self.h = h
        pass
    def f(self, x):
        return f(x)
    
    def g(self, x):
        return g(x)
    
    def hvp(self, x, g):
        return hvp(x, g)
    
obj = objective()

trlib.umin(obj.f, obj.g, obj.hvp, np.random.random(5))


it   obj         ‖g‖        radius     step       rho          ?  nhv
   0 +3.7011e+00 7.5428e+00 4.4721e-01 4.4721e-01 +1.0000e+00  +    1
   1 +1.1041e+00 4.0731e+00 8.9443e-01 5.2646e-01 +1.0000e+00  +    1
   2 +3.1979e-02 2.6923e-01 1.7889e+00 2.3309e-01 +1.0000e+00  +    2
   3 +7.3361e-04 5.4025e-02 3.5777e+00 3.0748e-02 +1.0000e+00  +    3
   4 +3.9704e-07 9.9020e-04 7.1554e+00 8.0639e-04 +1.0000e+00  +    4
   5 +4.2011e-15 9.1746e-08


array([  4.35879979e-08,   7.19599283e-09,  -2.80591440e-08,
         1.81572940e-08,  -7.29271157e-08])

In [39]:
import VariationalBayes as vb
from VariationalBayes.SparseObjectives import Objective
from copy import deepcopy

k = 3
par = vb.ModelParamsDict('par')
par.push_param(vb.VectorParam('a', size=k, lb=-5.0))

class StatModel(object):
    def __init__(self, par, h):
        self.h = h
        self.par = deepcopy(par)
        
    def obj(self):
        a = self.par['a'].get()
        return 0.5 * np.dot(a, np.matmul(h, a))

h = np.random.random((k, k))
h = np.matmul(h, h.transpose()) + np.eye(k)

stat_model = StatModel(par, h)

init_par = np.random.random(k)
stat_model.par.set_free(init_par)
stat_model.obj()

obj = Objective(stat_model.par, stat_model.obj)

obj.fun_free(init_par)
obj.fun_free_grad(init_par)
obj.fun_free_hvp(init_par, init_par)

trlib.umin(lambda par: obj.fun_free(par, verbose=True),
           obj.fun_free_grad, obj.fun_free_hvp, init_par)


it   obj         ‖g‖        radius     step       rho          ?  nhv
Iter  0  value:  41.198253167
Iter  1  value:  56.8599667796
Iter  2  value:  56.8599667796
   0 +5.6860e+01 2.4284e+01 5.7735e-01 5.7735e-01 +9.9662e-01  +    1
Iter  3  value:  7.24271732897
Iter  4  value:  41.198253167
Iter  5  value:  41.198253167
   1 +4.1198e+01 2.9817e+01 1.1547e+00 1.1547e+00 +8.5006e-01  +    1
Iter  6  value:  5.55779763135
Iter  7  value:  7.24271732897
Iter  8  value:  7.24271732897
   2 +7.2427e+00 1.9641e+01 1.1547e+00 1.1547e+00 +1.1466e-01  +    2
Iter  9  value:  0.817763127466
Iter  10  value:  5.55779763135
Iter  11  value:  5.55779763135
   3 +5.5578e+00 3.5490e+01 1.1547e+00 2.3234e-01 +1.1497e+00  +    1
Iter  12  value:  0.232873685065
Iter  13  value:  0.817763127466
Iter  14  value:  0.817763127466
   4 +8.1776e-01 8.5743e+00 2.3094e+00 1.2506e-01 +1.0910e+00  +    1
Iter  15  value:  0.00474328560605
Iter  16  value:  0.232873685065
Iter  17  value:  0.232873685065
   5 +2.

array([ 1.60943792,  1.60943791,  1.60943791])